# Part I: Algebraic Structures

In this section we define a `Polynomial2` class to represent polynomials over the field GF(2). Each instance carries a list of bits—one for each coefficient—starting from the constant term up to the highest power. For example, the list

    [0, 1, 1, 0, 0, 1]

encodes  
\[
x^5 + x^2 + x
\]

because the bit at index 5 is 1 (contributing \(x^5\)), at index 2 is 1 (contributing \(x^2\)), and at index 1 is 1 (contributing \(x\)).

## Key Methods

- **Constructor & Trimming**  
  On initialization, any trailing zeros are removed so that every polynomial is stored in its minimal form.

- **Addition / Subtraction**  
  Over GF(2), addition and subtraction are identical operations—both correspond to bitwise XOR of the coefficient vectors. For two polynomials \(p\) and \(q\),

\[
p + q = p - q = (p_0 \oplus q_0) + (p_1 \oplus q_1)x + \cdots
\]

These are exposed via `add()` and `sub()`, each returning a new `Polynomial2` instance.

- **Multiplication / Modular Reduction**  
  The `mul(p2, modp=None)` method first carries out a carry-less convolution of coefficient bits to form the product polynomial. If an irreducible modulus polynomial (`modp`) is provided, it then performs repeated XOR-based subtraction of the modulus—shifted appropriately—until the result has degree less than that of the modulus. This implements multiplication in \(\mathrm{GF}(2^n)\).

- **Division**  
  A classic long-division algorithm returns both quotient and remainder, again using only XOR and bit shifts.

---

# Part II: Building GF(2ⁿ)

Using `Polynomial2` as the backbone, we wrap field elements in a second class, `GF2N`, which:

1. **Encodes** an integer \(x\) as its bit-vector polynomial.  
2. **Reduces** automatically modulo an irreducible polynomial \(p(x)\) (if provided) to obtain a representative in \(\mathrm{GF}(2^n)\).  
3. Supplies field operations `add()`, `sub()`, and `mul()` by delegating to `Polynomial2` and then re-wrapping the result.  
4. (Optionally) provides multiplicative inverses via the extended Euclidean algorithm, and—for \(n=8\)—an AES-style affine map.

**Example usage:**

    a = GF2N(0b1101, 4, modulus)
    b = GF2N(0b0110, 4, modulus)
    c = a.mul(b)

performs a correct field multiplication in \(\mathrm{GF}(2^4)\).

---

# Part III: Generating the Tables

Finally, to illustrate the structure of \(\mathrm{GF}(2^4)\) concretely, we enumerate all 16 field elements and write out two 16×16 tables:

1. **Addition Table**: each entry \((i, j)\) shows \((i \oplus j)\).  
2. **Multiplication Table**: each entry shows the reduced product \((i \cdot j) \bmod (x^4 + x^3 + 1)\).

These tables are saved to `table1.txt` for submission. By inspecting this file, one can verify any entry by hand using the polynomial operations defined above.


In [2]:
!python3 gf2n.py



Test 1
p1=x^5+x^2+x
p2=x^3+x^2+1
p3= p1+p2 = x^5+x^3+x+1

Test 2
p4=x^7+x^4+x^3+x^2+x
modp=x^8+x^7+x^5+x^4+1
p5=p1*p4 mod (modp)= x^7+x^6+x^4+x^3

Test 3
p6=x^12+x^7+x^2
p7=x^8+x^4+x^3+x+1
q for p6/p7= x^4+1
r for p6/p7= x^5+x^3+x^2+x+1

Test 4
g1 = x^6+x^5+x^2
g2 = x^2+1
g1+g2 = x^6+x^5+1

Test 5
irreducible polynomial x^4+x+1
g4 = x^3+x^2+1
g5 = x^2+x
g4 x g5 = x^3

Test 6
g7 = x^12+x^7+x^2
g8 = x^8+x^4+x^3+x+1
g7/g8 =
q = x^4+1
r = x^5+x^3+x^2+x+1

Test 7
irreducible polynomial x^4+x+1
g9 = x^2+1
inverse of g9 = x^3+x+1

Test 8
irreducible polynomial x^8+x^4+x^3+x+1
g10 = 0xc2
inverse of g10 = g11 = 0x2f
affine map of g11 = 0x25

Generated table1.txt with GF(2^4) addition and multiplication tables.


In [3]:
!cat table1.txt

Addition Table (16×16)
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
1 0 3 2 5 4 7 6 9 8 11 10 13 12 15 14
2 3 0 1 6 7 4 5 10 11 8 9 14 15 12 13
3 2 1 0 7 6 5 4 11 10 9 8 15 14 13 12
4 5 6 7 0 1 2 3 12 13 14 15 8 9 10 11
5 4 7 6 1 0 3 2 13 12 15 14 9 8 11 10
6 7 4 5 2 3 0 1 14 15 12 13 10 11 8 9
7 6 5 4 3 2 1 0 15 14 13 12 11 10 9 8
8 9 10 11 12 13 14 15 0 1 2 3 4 5 6 7
9 8 11 10 13 12 15 14 1 0 3 2 5 4 7 6
10 11 8 9 14 15 12 13 2 3 0 1 6 7 4 5
11 10 9 8 15 14 13 12 3 2 1 0 7 6 5 4
12 13 14 15 8 9 10 11 4 5 6 7 0 1 2 3
13 12 15 14 9 8 11 10 5 4 7 6 1 0 3 2
14 15 12 13 10 11 8 9 6 7 4 5 2 3 0 1
15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0

Multiplication Table (16×16)
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
0 2 4 6 8 10 12 14 9 11 13 15 1 3 5 7
0 3 6 5 12 15 10 9 1 2 7 4 13 14 11 8
0 4 8 12 9 13 1 5 11 15 3 7 2 6 10 14
0 5 10 15 13 8 7 2 3 6 9 12 14 11 4 1
0 6 12 10 1 7 13 11 2 4 14 8 3 5 15 9
0 7 14 9 5 2 11 12 10 13 4 3 15 8 1 6
0 8 9 1 11 3 2 10 15 7 6 14 4 12 13 5
0 9

# Part IV: Quiz

There is no difference between addition and subtraction in GF(3).
False. In GF(3) = ℤ/3ℤ, subtraction is a–b = a + (–b), and since –b ≠ b in general (for example 1–2 = 1+1 = 2, whereas 1+2 = 0), the two operations differ.

There is no difference between addition and subtraction in GF(2ⁿ), for any integer n.
True. All fields of characteristic 2 satisfy –a = a, so a–b = a + (–b) = a + b; thus subtraction and addition coincide.

For all elements of GF(2), multiplication is equivalent to an AND gate.
True. GF(2) = {0,1} with 1·1 = 1 and 1·0 = 0·0 = 0 matches the truth table of logical AND.

A polynomial of degree 8 is a possible element of the field GF(2⁸).
False. Elements of GF(2⁸) are represented by polynomials of degree ≤ 7 (i.e. degree < 8); any degree-8 polynomial is reduced modulo the irreducible polynomial down to degree ≤ 7.

P(x) = x⁴ + x³ + x + 1 is an irreducible polynomial.
False. P(1) = 1+1+1+1 = 0 in GF(2), so x = 1 is a root ⇒ P(x) is divisible by (x+1) and thus reducible.